In [1]:
from sfm.data.sci_data.SFMDecTokenizer import SFMDecTokenizer
import torch
import os

/anaconda/envs/sfm2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-03 13:25:09,347] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
tokenizer_home = '/hai1/ds_dataset/llama2/llama-2-7b'
tokenizer = SFMDecTokenizer.from_pretrained(
    tokenizer_home,
    prot_spm_path='/blob/shufxi/data/scigpt/ur50bpe/bpe',
    dna_spm_path='/blob/shufxi/data/scigpt/dnabpe/bpe',
    rna_spm_path='/blob/shufxi/data/scigpt/rnabpe/bpe',
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'SFMDecTokenizer'.
You are using the default legacy behaviour of the <class 'sfm.data.sci_data.SFMDecTokenizer.SFMDecTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


[2024-03-03 13:25:12.034][INFO]: Loading protein sentencepiece model from /blob/shufxi/data/scigpt/ur50bpe/bpe.model and /blob/shufxi/data/scigpt/ur50bpe/bpe.vocab
[2024-03-03 13:25:12.700][INFO]: Loading DNA sentencepiece model from /blob/shufxi/data/scigpt/dnabpe/bpe.model and /blob/shufxi/data/scigpt/dnabpe/bpe.vocab
[2024-03-03 13:25:12.939][INFO]: Loading RNA sentencepiece model from /blob/shufxi/data/scigpt/rnabpe/bpe.model and /blob/shufxi/data/scigpt/rnabpe/bpe.vocab
[2024-03-03 13:25:13.436][INFO]: Tokenizer has 40014 tokens


In [3]:
tokenizer.tokenize('<rna>GCCGGCGUAGCUCAGUUGGUAGAGCAAUUGUUUUGUAAACAAAAGGUCGGGGGUUCGAUUCCUCUCGCCGGCU</rna>')

['<rna>',
 '<r>GCCG',
 '<r>GCGU',
 '<r>AGCUC',
 '<r>AGUUGG',
 '<r>UAG',
 '<r>AGCAAU',
 '<r>UGUU',
 '<r>UUGUAAA',
 '<r>CAAA',
 '<r>AGGU',
 '<r>CGGG',
 '<r>GGUUCGAUUCC',
 '<r>UCUC',
 '<r>GCC',
 '<r>GGCU',
 '</rna>']

In [4]:
s = tokenizer.tokenize('The <protein>EMMFEQTFKNID</protein> and the compound <mol>CCC</mol> can interact with each other.')
print(' '.join(s))

▁The <protein> <a>E <a>M <a>M <a>FEQ <a>TFK <a>NID </protein> ▁and ▁the ▁comp ound <mol> <m>C <m>C <m>C </mol> ▁can ▁interact ▁with ▁each ▁other .


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# ckpt_home = '/hai1/shufxi/scigpt/7bv2/stageB/global_step26999/'

# ckpt_home = '/hai1/shufxi/scigpt/7bv3/stageB/global_step11999'
# ckpt_home = '/blob/shufxi/scigpt/7bv3/inst/20240227121523/global_step3585/'
ckpt_home = '/blob/shufxi/scigpt/7bv3/prot/20240228025826/global_step11715'

def show_ckpt(name, ckpt):
    for k, v in ckpt.items():
        if 'dummy' not in k:
            print(name, k, v.shape)

model = AutoModelForCausalLM.from_pretrained(tokenizer_home)

model_dict = model.state_dict()
ckpt_dict = {}
layer0 = torch.load(os.path.join(ckpt_home, "layer_00-model_states.pt"), map_location=torch.device("cpu"))
ckpt_dict['model.embed_tokens.weight'] = layer0['embed_tokens.weight']
show_ckpt('layer0', layer0)

for l in range(0, 32):
    l_index = str(l + 1).zfill(2)
    layer = torch.load(os.path.join(ckpt_home, f"layer_{l_index}-model_states.pt"), map_location=torch.device("cpu"))
    show_ckpt(l_index, layer)
    for k in layer:
        if "dummy" in k or 'rotary_emb' in k:
            continue
        ckpt_dict[f"model.layers.{l}.{k}"] = layer[k]
layer = torch.load(os.path.join(ckpt_home, "layer_33-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["model.norm.weight"] = layer["norm.weight"]

layer = torch.load(os.path.join(ckpt_home, "layer_34-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["lm_head.weight"] = layer["lm_head.weight"]
model_dict.update(ckpt_dict)

model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(model_dict)

Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.02s/it]


layer0 embed_tokens.weight torch.Size([40014, 4096])
01 self_attn.q_proj.weight torch.Size([4096, 4096])
01 self_attn.k_proj.weight torch.Size([4096, 4096])
01 self_attn.v_proj.weight torch.Size([4096, 4096])
01 self_attn.o_proj.weight torch.Size([4096, 4096])
01 mlp.gate_proj.weight torch.Size([11008, 4096])
01 mlp.up_proj.weight torch.Size([11008, 4096])
01 mlp.down_proj.weight torch.Size([4096, 11008])
01 input_layernorm.weight torch.Size([4096])
01 post_attention_layernorm.weight torch.Size([4096])
02 self_attn.q_proj.weight torch.Size([4096, 4096])
02 self_attn.k_proj.weight torch.Size([4096, 4096])
02 self_attn.v_proj.weight torch.Size([4096, 4096])
02 self_attn.o_proj.weight torch.Size([4096, 4096])
02 mlp.gate_proj.weight torch.Size([11008, 4096])
02 mlp.up_proj.weight torch.Size([11008, 4096])
02 mlp.down_proj.weight torch.Size([4096, 11008])
02 input_layernorm.weight torch.Size([4096])
02 post_attention_layernorm.weight torch.Size([4096])
03 self_attn.q_proj.weight torch.Size

<All keys matched successfully>

In [6]:
model = model.cuda()

In [11]:
input_ids = tokenizer('<protein>', return_tensors="pt").input_ids.cuda()
output = model.generate(
    input_ids,
    num_beams=4,
    max_new_tokens=300,
    num_return_sequences=4,
    return_dict_in_generate=True,
    output_scores=True,
    do_sample=True,
    top_p=0.95,
    repetition_penalty=1.5
)

for i in range(output.sequences.shape[0]):
    s = tokenizer.decode(output.sequences[i])
    s = s.replace(' <a>', '').replace(' <m>', '')
    print(s)
    print("=====================================")

<s> <protein>MSSMIIAPKKDQVQTEGDPCWVLPELPFKGRLELTKEQLIENIEKIGREEFQYRRDSHRELKNSCCISLHISEKDDFK </protein> </s>
<s> <protein>MSSMLILRWRFAFPEAGRHAAAQRYEYPQKHGPLTPGHGQQYVDQGEEGVAENEGDEEGFTRDIEQRFPSW </protein> </s><unk>
<s> <protein>MSETRPARRPTRLVRLLVGVVVTIVVVWLLFEFVFPWVESRWYNPAVGVVTWWI </protein> </s><unk><unk><unk><unk><unk><unk><unk><unk>
<s> <protein>MSSMIIAPKKDQVQTEGDPCWVLPELPFKGRLELTKEQLIENIEKIGREEFQYRRDSHRELKNSCCISLHISEKDDIK </protein> </s>


In [10]:
def gen_seq():
    input_ids = tokenizer('<protein>', return_tensors="pt").input_ids.cuda()
    output = model.generate(
        input_ids,
        num_beams=4,
        max_new_tokens=300,
        num_return_sequences=1,
        return_dict_in_generate=True,
        output_scores=True,
        do_sample=True,
        top_p=0.90,
    )

    s = tokenizer.decode(output.sequences[0])
    start_idx = s.find('<protein>') + len('<protein>')
    end_idx = s.find('</protein>')
    s = s[start_idx:end_idx].replace(' <a>', '')
    return s

gen_seq()

'MSRDWLLSQFGATRDEAVANYSRFVAAGMGETSPLAHTCHQMLLGDDAFVSAHQKSQRSNAFKDAPRQQRRAVTLSLAQYQARYSDRDMAMARAYLSTAFTMSHIAAAFHVSS '

In [11]:
from tqdm import tqdm

In [ ]:
with open('prot_gen.txt', 'w') as f:
    for i in tqdm(range(3000)):
        s = gen_seq()
        f.write(s + '\n')

In [ ]:
input_ids = tokenizer('Instruction: Analyze and describe the functional role of the corresponding protein for the sequence <protein>MTEFESAPAYQEAKYLTSAAEFDQLPPDQGAEIAFIGRSNAGKSSALNIITGIKGLARTSKTPGRTQMINFFALNEHERLVDLPGYGYAKVPRMVQKRWEELVDSYLKKRRCLKGLVVVMDIRHPLKEMDEDVIEWAVNYDIPIHILLTKSDKLSQNAAKKTLGEVQTAISAYGEKLTLQLFSSHDRTGLDEVKAVLSQWFRSEP</protein>.\n\n\nResponse:', return_tensors="pt").input_ids.cuda()
output = model.generate(
    input_ids,
    num_beams=4,
    max_new_tokens=300,
    num_return_sequences=4,
    return_dict_in_generate=True,
    output_scores=True,
    do_sample=True,
    top_p=0.95,
)

for i in range(output.sequences.shape[0]):
    s = tokenizer.decode(output.sequences[i])
    s = s.replace(' <a>', '').replace(' <m>', '')
    print(s)
    print("=====================================")

In [ ]:
import random

In [ ]:
def chat(instruction: str, beam:int=4, do_sample:bool=True, top_p:float=0.9):
    prompt = f"Instruction: {instruction}\n\n\nResponse:"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    output = model.generate(
        input_ids,
        num_beams=beam,
        max_new_tokens=50,
        num_return_sequences=1,
        return_dict_in_generate=True,
        output_scores=True,
        do_sample=do_sample,
        top_p=top_p,
        repetition_penalty=2.0
    )
    ret = tokenizer.decode(output.sequences[0])
    start_idx = ret.find('Response:') + len('Response:')
    end_idx = ret.find('</s>')

    ret = ret[start_idx:end_idx]
    ret = ret.replace(' <m>', '').replace(' <a>', '')

    return ret

In [ ]:
chat('design a transmembrane protein')

In [ ]:
chat('design a transmembrane protein')

In [ ]:
chat('design a protein that fold fast')

In [ ]:
def compute_ppl(seq):
    input_ids = tokenizer(seq, return_tensors="pt").input_ids.cuda()
    labels = input_ids.clone()

    with torch.no_grad():
        outputs = model(input_ids, labels=labels, return_dict=True) # shift inside
    loss = outputs.loss
    return loss.item(), torch.exp(loss).item()

compute_ppl('<mol>CCN(CC)CCCC(C)Nc1ccnc2cc(Cl)ccc12</mol>')

In [ ]:
print(compute_ppl('<protein>MEFLGLW'))
print(compute_ppl('<protein>MEFLGM'))
print(compute_ppl('<protein>MTTCRISQTARFNLPGIIAFESAKRGGELLDVPDLGNPPEDWEMLNPDGEYGTNPVVDEIVKKS</protein>'))

In [ ]:
torch.softmax(output.scores[0][0], dim=0).sort(descending=True)

In [ ]:
output.scores[0].shape

In [ ]:
output.sequences[0]

In [ ]:
input_ids = tokenizer('An RNA like <rna>', return_tensors="pt").input_ids.cuda()
n = 4
output = model.generate(
    input_ids,
    num_beams=n,
    do_sample=True,
    max_new_tokens=100,
    num_return_sequences=n,
    return_dict_in_generate=True,
    output_scores=True,
    top_k=10
)

for i in range(n):
    s = tokenizer.decode(output.sequences[i])
    # print(s)
    # mol = s[s.find('<rna>') + len('<rna>'):s.find('</rna>')]
    s = s.replace('<r>', '')
    print(s)

In [ ]:
input_ids = tokenizer('<rna>GCCGGCGUAGCUCAGUUGG', return_tensors="pt").input_ids.cuda()
n = 10
output = model.generate(
    input_ids,
    num_beams=n,
    do_sample=False,
    # top_p=0.95,
    max_new_tokens=100,
    num_return_sequences=n,
    return_dict_in_generate=True,
    output_scores=True
)

for i in range(n):
    s = tokenizer.decode(output.sequences[i])
    s = s.replace('<r>', '').replace(' ', '')
    print(s)

In [ ]:
input_ids = tokenizer('Elements, energy above hull and band gap to material: The material contains Ba, B, Pb, O. Its energy above hull is 0.01 eV/atom. Its band gap is 4.1 eV. The material is <material>', return_tensors="pt").input_ids.cuda()
n = 4
output = model.generate(
    input_ids,
    num_beams=n,
    do_sample=False,
    max_new_tokens=200,
    num_return_sequences=n,
    return_dict_in_generate=True,
    output_scores=True,
)

for i in range(n):
    s = tokenizer.decode(output.sequences[i])
    # print(s)
    # mol = s[s.find('<rna>') + len('<rna>'):s.find('</rna>')]
    s = s.replace('<i>', '')
    print(s)

In [ ]:
chat('design a protein that folds fast')